In [22]:
import hashlib
import json

import matplotlib.pyplot as plt
import numpy as np
import requests
from bs4 import BeautifulSoup
from bs4.element import Tag
from tqdm import tqdm

from functools import partial
import random
import os

In [21]:
with open("../templates/comparing.json", "r") as f:
    comparasions = json.load(f)

In [3]:
comparasions[random.randint(0, len(comparasions) - 1)]

[['Rainbow Dash Facehoof (MLP)', 3856],
 ['punch to death', 570],
 41,
 {'simple_r': 29, 'token_sort_r': 41}]

In [4]:
# ratios = [comp[2] for comp in comparasions]

In [5]:
# len(np.unique(ratios))

In [6]:
# plt.hist(ratios, bins=25)
# plt.xlabel("Comparion score (0-100)")
# plt.ylabel("Count")
# plt.title("Distribution of comparasion scores")
# plt.savefig("Distribution of comparasion scores.png")

In [7]:
def custom_r(data):
    return data[2]
    # compares = list(data[3].values())
    # return max(compares)


def compare_func(data: list, threshold: int) -> bool:
    r = custom_r(data)
    return r >= threshold


def filter_data(data: list, threshold: int) -> list:
    """
    Return a sublist with comparasion score >= threshold
    """
    f = partial(compare_func, threshold=threshold)
    return list(filter(f, data))

In [8]:
filtered = filter_data(comparasions, 86)
filtered.sort(key=lambda x: custom_r(x))
len(filtered)

798

In [9]:
len(filtered), [(x[0][0], x[1][0], custom_r(x)) for x in filtered]

(798,
 [('no god please no meme', 'No god please no', 86),
  ('Man Sweating', 'Man Sweating Gif', 86),
  ('Cat fight', 'Fighting cat', 86),
  ('TPOT Two Gets Beaten Up', 'Two gets beaten up (HD)', 86),
  ('CPR CAT', 'cat cry', 86),
  ('911', '9/11', 86),
  ('Walter White screaming in car', 'Walter White screaming', 86),
  ("Get In Loser We're Going Shopping - Mean Girls",
   "Get in loser we're going shopping",
   86),
  ('Fast Talking', 'TALK FAST', 86),
  ('Average Fan VS. Average Enjoyer', 'Average fan vs Average fan', 86),
  ('Back on the menu 2', 'is back on the menu', 86),
  ('Average Fan vs Average Enjoyer', 'Average fan vs Average fan', 86),
  ('Cat fight', 'Fighting cat', 86),
  ('SpongeBob bow', 'Bored Spongebob', 86),
  ('Average Fan vs Average Enjoyer', 'Average fan vs Average fan', 86),
  ('Fast Talk', 'Fast Talking', 86),
  ('Dissapearing', 'Guy dissapearing', 86),
  ('Typing Jim Carrey GIF', 'Jim Carry Typing', 86),
  ('Dissapear', 'Dissapearing', 86),
  ('Confused Dog G

In [10]:
merged = []  # list of (set of names, set of index)
for i in tqdm(range(len(filtered))):
    t1, t2, _, _ = filtered[i]

    if_existed = False
    for m in merged:
        if t1[0] in m[0] or t2[0] in m[0]:
            if_existed = True
            # add title
            m[0].add(t1[0])
            m[0].add(t2[0])

            # add index
            m[1].add(t1[1])
            m[1].add(t2[1])

    if not if_existed:
        merged.append((set([t1[0], t2[0]]), set([t1[1], t2[1]])))

100%|████████████████████████████████████████████████| 798/798 [00:00<00:00, 64998.92it/s]


In [11]:
len(merged), merged

(407,
 [({'No god please no', 'no god please no meme'}, {22, 78}),
  ({'Man Sweating', 'Man Sweating Gif'}, {225, 563}),
  ({'Cat fight', 'Fighting cat', 'Rat Fight'}, {903, 967, 2487, 2960}),
  ({'TPOT Two Gets Beaten Up',
    'Two Gets Beaten Up',
    'Two gets beaten up (HD)'},
   {753, 1144, 4281}),
  ({'CPR CAT', 'cat cry'}, {357, 1179}),
  ({'9/11', '911'}, {759, 1370}),
  ({'Walter White screaming', 'Walter White screaming in car'}, {205, 1664}),
  ({"Get In Loser We're Going Shopping - Mean Girls",
    "Get in loser we're going shopping"},
   {2009, 2107}),
  ({'Fast Talk', 'Fast Talking', 'TALK FAST', 'Talking fast'},
   {481, 2129, 2862, 4741}),
  ({'Average Fan VS. Average Enjoyer',
    'Average Fan vs Average Enjoyer',
    'Average fan vs Average fan',
    'avarage fan vs avarage enjoyer',
    'average fan vs average enjoyer'},
   {761, 980, 2162, 2441, 2780, 4828}),
  ({'Back on the menu 2', 'is back on the menu'}, {927, 2180}),
  ({'Bored Spongebob', 'SpongeBob bow'}, {15

In [18]:
with open("../templates/metadata_w_examples.json") as f:
    metadatas = json.load(f)

In [19]:
len(metadatas)

5113

In [14]:
metadatas[0]

{'href': '/memetemplate/173865152/Eyebrow-Spock',
 'title': 'Eyebrow Spock',
 'html_file': '-memetemplate-173865152-Eyebrow-Spock',
 'src': 'i.imgflip.com/2vij7k.mp4',
 'src_type': 'video/mp4',
 'Template ID': '173865152',
 'Format': 'mp4',
 'Dimensions': '220x270 px',
 'Filesize': '17 KB',
 'examples': [{'title': "It's that easy",
   'url': 'https://i.imgflip.com/7ztpy9.gif'},
  {'title': 'corona', 'url': 'https://i.imgflip.com/3sc1a5.gif'}]}

In [15]:
# merge examples
# change example file name

In [16]:
directory = "../templates/template_examples"

print("Before:", len(metadatas))

delete_indexes = []
for merge in merged:
    indexes = list(merge[1])
    og_metadata = metadatas[indexes[0]]
    og_template_id = og_metadata["Template ID"]
    for i in range(len(merge[1])):
        if i == 0:
            continue

        index = indexes[i]
        metadata = metadatas[index]
        delete_indexes.append(index)

        template_id = metadata["Template ID"]
        for example in metadata["examples"]:
            og_metadata["examples"].append(example)

            byte_string = example["title"].encode()
            md5_hash = hashlib.md5(byte_string).hexdigest()
            ext = example["url"].split("/")[-1].split(".")[-1]
            old_filename = f"{md5_hash}_{template_id}.{ext}"
            new_filename = f"{md5_hash}_{og_template_id}.{ext}"

            old_file_path = os.path.join(directory, old_filename)
            new_file_path = os.path.join(directory, new_filename)
            if os.path.exists(old_file_path):
                os.rename(old_file_path, new_file_path)
                print(f"File renamed from {old_file_path} to {new_file_path}")
            else:
                print("Not exist: ", old_filename)

for i in sorted(delete_indexes, reverse=True):
    del metadatas[i]

print("After:", len(metadatas))

Before: 5704
File renamed from ../templates/template_examples/2fbd38e6c6c4a64ef43fac3f0be7860e_390413536.gif to ../templates/template_examples/2fbd38e6c6c4a64ef43fac3f0be7860e_476220766.gif
File renamed from ../templates/template_examples/085112f7031760bea52485f364d26d6e_496242785.gif to ../templates/template_examples/085112f7031760bea52485f364d26d6e_392839501.gif
File renamed from ../templates/template_examples/4ba2b26f7ef90563b758e577b7448d27_291163792.gif to ../templates/template_examples/4ba2b26f7ef90563b758e577b7448d27_392839501.gif
File renamed from ../templates/template_examples/e6f48a2782e460e29f7722c337df0ab2_291163792.gif to ../templates/template_examples/e6f48a2782e460e29f7722c337df0ab2_392839501.gif
File renamed from ../templates/template_examples/84ff8a71afc4e3a1ce526dd9b9da4c16_291163792.gif to ../templates/template_examples/84ff8a71afc4e3a1ce526dd9b9da4c16_392839501.gif
File renamed from ../templates/template_examples/4ce60b425328ada090b575599f7d8257_291163792.gif to ../

In [17]:
# with open("../templates/metadata_w_examples.json", "w") as f:
#     json.dump(metadatas, f, indent = 6)

In [25]:
for merge in merged:
    # if "Bunny jumping into trash can" in merge[0]:
    print(merge[0])

{'no god please no meme', 'No god please no'}
{'Man Sweating', 'Man Sweating Gif'}
{'Rat Fight', 'Fighting cat', 'Cat fight'}
{'Two Gets Beaten Up', 'TPOT Two Gets Beaten Up', 'Two gets beaten up (HD)'}
{'CPR CAT', 'cat cry'}
{'911', '9/11'}
{'Walter White screaming in car', 'Walter White screaming'}
{"Get in loser we're going shopping", "Get In Loser We're Going Shopping - Mean Girls"}
{'TALK FAST', 'Fast Talking', 'Talking fast', 'Fast Talk'}
{'average fan vs average enjoyer', 'avarage fan vs avarage enjoyer', 'Average fan vs Average fan', 'Average Fan VS. Average Enjoyer', 'Average Fan vs Average Enjoyer'}
{'is back on the menu', 'Back on the menu 2'}
{'SpongeBob bow', 'Bored Spongebob'}
{'Guy dissapearing', 'guy disappearing', 'Dissapearing', 'dissapearing guy', 'disappear', 'Dissapear'}
{'Typing Jim Carrey GIF', 'Jim carrey typing', 'Jim Carry Typing'}
{'Confused dog', 'Confused Dog GIF', 'confused dog'}
{'On my way', 'on my way', 'me on my way'}
{'MGR Raiden punching Armstrong', 